In [6]:
import pickle

images, pca_features, pca = pickle.load(open('data/features_caltech101.p', 'rb'))

for i, f in list(zip(images, pca_features))[0:5]:
    print("image: %s, features: %0.2f,%0.2f,%0.2f,%0.2f... "%(i, f[0], f[1], f[2], f[3]))

image: data/101_ObjectCategories/accordion/image_0001.jpg, features: -7.55,-6.28,-21.75,5.71... 
image: data/101_ObjectCategories/accordion/image_0002.jpg, features: -8.51,-8.37,-17.40,-4.45... 
image: data/101_ObjectCategories/accordion/image_0003.jpg, features: -11.12,-10.11,-21.06,7.69... 
image: data/101_ObjectCategories/accordion/image_0004.jpg, features: -12.16,-11.77,-21.84,7.25... 
image: data/101_ObjectCategories/accordion/image_0005.jpg, features: -9.15,-3.14,-13.55,-0.50... 


In [7]:
num_images = 10000

if len(images) > num_images:
    sort_order = sorted(random.sample(xrange(len(images)), num_images))
    images = [images[i] for i in sort_order]
    pca_features = [pca_features[i] for i in sort_order]

In [9]:

from igraph import *

kNN = 30

graph = Graph(directed=True)
graph.add_vertices(len(images))

for i in tqdm(range(len(images))):
    distances = [ distance.cosine(pca_features[i], feat) for feat in pca_features ]
    idx_kNN = sorted(range(len(distances)), key=lambda k: distances[k])[1:kNN+1]
    for j in idx_kNN:
        graph.add_edge(i, j, weight=distances[j])
    
summary(graph)

Traceback (most recent call last):
  File "/usr/local/bin/pip", line 5, in <module>
    from pkg_resources import load_entry_point
  File "/usr/local/lib/python2.7/site-packages/distribute-0.6.14-py2.7.egg/pkg_resources.py", line 2671, in <module>
    working_set.require(__requires__)
  File "/usr/local/lib/python2.7/site-packages/distribute-0.6.14-py2.7.egg/pkg_resources.py", line 654, in require
    needed = self.resolve(parse_requirements(requirements))
  File "/usr/local/lib/python2.7/site-packages/distribute-0.6.14-py2.7.egg/pkg_resources.py", line 552, in resolve
    raise DistributionNotFound(req)
pkg_resources.DistributionNotFound: pip==9.0.1


ImportError: No module named 'igraph'

In [ ]:
pickle.dump([images, graph], open('data/graph_caltech101_30knn.p', 'wb'))
#images, graph = pickle.load(open('../data/graph_caltech101_30knn.p', 'r'))

In [ ]:
def get_concatenated_images(indexes, thumb_height):
    thumbs = []
    for idx in indexes:
        img = Image.open(images[idx])
        img = img.convert('RGB')
        img = img.resize((int(img.width * thumb_height / img.height), thumb_height))
        thumbs.append(img)
    concat_image = np.concatenate([np.asarray(t) for t in thumbs], axis=1)
    return concat_image

In [ ]:
# pick two random indices
idx1 = int(len(images) * random.random())
idx2 = int(len(images) * random.random())

# run get_shortest_paths
path = graph.get_shortest_paths(idx1, to=idx2, mode=OUT, output='vpath', weights='weight')[0]

# retrieve the images, concatenate into one, and display them
results_image = get_concatenated_images(path, 200)
matplotlib.pyplot.figure(figsize = (16,12))
imshow(results_image)